# We want to create a synthetic time series data set

> Based on the synthetic data generation notebook of the ```afa``` repository by Henrik von Kleist.

In [6]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pathlib
relative_path_to_afa = '../../afa_github/afa'
#sys.path.insert(0, os.path.abspath(relative_path_to_afa))
sys.path.insert(1, os.path.join(sys.path[0], relative_path_to_afa))
print('CWD:', pathlib.Path(pathlib.Path.cwd()))

import sklearn

# data generation
from afa_datasets.synthetic_data.data_generation_ts import generate_data_ts

# loading/ saving
from data_modelling.clairvoyance_static.datasets.data_loader import CSVLoader, save_dataset
from data_modelling.clairvoyance_static.utils.model_utils import PipelineComposer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CWD: /home2/joshua.wendland/Documents/sepsis/toy_dataset


In [ ]:
# define directories
data_dir = "dataset/"
dataset_name = "synthetic_1"

# Datageneration

Use sklearn's make-classification function

In [26]:
# df_train, df_test = generate_data_ts(data_dir = data_dir + dataset_name + "/", 
#                                          dataset_name = dataset_name,
#                                          n_features_static  = 4 ,
#                                          n_features_ts  = 10 ,
#                                          n_redundant = 0 ,
#                                          n_informative = 2,
#                                          n_datapts   = 200000,
#                                          n_classes   = 2,
#                                          random_state  = 47,
#                                          test_size   = 0.5)

data_train_ts_df, data_train_static_df, data_test_ts_df, data_test_static_df = generate_data_ts(data_dir = "",
                                        dataset_name = "synthetic_ts_1",
                                        n_features_static  = 2  ,
                                        n_features_ts      = 4  , 
                                        n_timepts   = 50,
                                        n_datapts   = 100, 
                                        test_size = 0.1,
                                        table_style = 'wide',
                                        n_classes   = 2)